In [4]:
from ipynb.fs.full.Funciones_basicas import *

## greedyApp

- input: Grafo (G), lista de nodos visitados (V_), cantidad de nodos (N), Cantidad de muestras (K), límite de nodos por ideal (limit) y límite de personas por ideal (wlim)
- output: Lista con los K nodos a muestrear

In [7]:
def greedyApp(G, W, V_, K, limit, wlim):
    
    V = V_.copy(); N = len(V)
    
    E = []
    for u in range(N):
        if not V[u]:
            E.append((get_size(G, V, u), u))

    S = set()
    for e in E:
        if e[0] > 0 and e[0] <= limit:
            S.add(e[1])

    ans = []
    while len(ans) < K:
        maxv = 0; u = -1
        for v in S:
            s, w = get_size_weight(G, W, V, v)
            if s > maxv and w <= wlim:
                u = v
                maxv = s
        if u == -1:
            break

        V, _ = visit(G, V, u)
        ans.append(u)
        S.remove(u)

    return ans

## greedyAppReduce

- input: Grafo (G), Lista de pesos (W), lista de nodos visitados (V_), Cantidad de muestras (K), límite de nodos en la solución (limit) y límite de personas en la solución (wlim)
- output: Lista con K nodos a muestrear, Nodos en union de ideales a muestrear. Calcula ideales agregando nodos con menos de 200 personas en su ideal.

In [9]:
def greedyAppReduce(G, W, V_, K, limit, wlim):
    
    V = V_.copy(); N = len(V)
    
    E = []
    for u in range(N):
        if not V[u]:
            E.append((get_size(G, V, u), u))

    limit_set = set()
    for e in E:
        if e[0] > 0 and e[0] <= limit:
            limit_set.add(e[1])

    ans = []; tot = 0
    while len(ans) < K:
        
        S = [0 for u in range(N)]
        for v in limit_set:
            _, S[v] = get_size_weight(G, W, V, v)
        
        maxv = 0; u = -1
        for v in limit_set:
            s = get_size_precalc(G, S, V, v)
            if s > maxv and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break

        V, x = visit(G, V, u)
        ans.append(u); tot += x
        limit_set.remove(u)
        
    return ans, tot

## particionar
- Input: Grafo (G), lista de pesos (W), lista de visitados (V), nodo (node)
- Output: nodo que particiona el ideal

## partition_ideal
- Input: Grafo (G), lista de pesos (W), lista de visitados (V), nodo (node)
- Output: nodo que particiona el ideal y su ideal inducido

In [11]:
def partition(G, W, V_, node):
    # Obtenemos ideal y definimos nodos ya visitados 

    I = set(get_ideal(G, V_, node)); N = len(V_)
    V = [0 if i in I else 1 for i in range(N)]
    
    # Hacemos una búsqueda binaria para obtener la mejor partición
    R = get_size(G, V, node)
    low = 0; high = R
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppReduce(G, W, V, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppReduce(G, W, V, 1, low, 100000000)
    return P

def partition_ideal(G, W, V_, node):
    # Obtenemos ideal y definimos nodos ya visitados 
    I = set(get_ideal(G, V_, node)); N = len(V_)
    V = [0 if i in I else 1 for i in range(N)]
    
    # Hacemos una búsqueda binaria para obtener la mejor partición
    R = get_size(G, V, node)
    low = 0; high = R
    while low != high:
        mid = (low + high) // 2
        P, sP = greedyAppReduce(G, W, V, 1, mid, 100000000)
        if R - sP <= mid:
            high = mid
        else:
            low = mid + 1
    P, sP = greedyAppReduce(G, W, V, 1, low, 100000000)
    return P, get_ideal(G, V, P[0])

## greedyAppReduceDynamic

- Input: Grafo (G), Lista de pesos (W), lista de nodos visitados (V_), Cantidad de nodos (N), Cantidad de muestras (K), límite de nodos en la solución (limit) y límite de personas en la solución (wlim)
- Output: Lista con las K soluciones, Tamaño de la union de los ideales elegidos

In [12]:
def greedyAppReduceDynamic(G, W, V_, K, limit, wlim):
    
    V = V_.copy(); N = len(V)
    E = {u for u in range(N) if not V[u]}

    ans = []; tot = 0
    while len(ans) < K:
        
        S = [0 for u in range(N)]
        for v in E:
            _, S[v] = get_size_weight(G, W, V, v)
        
        maxv = 0; u = -1
        for v in E:
            s = get_size_precalc(G, S, V, v)
            if s > maxv and s <= limit and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break
         
        V, x = visit(G, V, u)
        ans.append(u); tot += x
        E.remove(u)
        
    return ans, tot

def greedyAppReduceDynamic_partition(G, W, V_, K, limit, wlim):
    
    V = V_.copy(); N = len(V)
    E = {u for u in range(N) if not V[u]}


    ans = []; divided = []; solution_dict = {}; precalc = {}
    tot = 0; sizes = {}; ideals = {}
    while len(ans) < K:
        
        S = [0 for u in range(N)]
        for v in E:
            _, S[v] = get_size_weight(G, W, V, v)
        
        maxv = 0; u = -1
        for v in E:
            s = get_size_precalc(G, S, V, v)
            if s > maxv and s <= limit and S[v] <= wlim:
                u = v
                maxv = s
        if u == -1:
            break
        
        # Comparamos la solución obtenida bajo el algoritmo glotón versus la opción de particionar uno de los ideales
        # de los nodos ya calculados
        moved = 0
        for node in list(set(ans) - set(divided)):
            ideal_size, ideal, s, ideal_partition  = precalc[node]; size_partition = len(ideal_partition) 
            min_partition = min(size_partition, ideal_size - size_partition)
            if min_partition > len(get_ideal(G, V, u)):
                u = s
                divided.append(node); divided.append(u); moved = 1;
                solution_dict[node] = [u]
                sizes[u] = size_partition; sizes[node] = ideal_size - size_partition
                ideals[u] = ideal_partition
                ideals[node] = [i for i in ideals[node] if i not in ideal_partition]
                break
                
        ideal = get_ideal(G, V, u); ideal_size = len(ideal)       
        if not moved:
            E.remove(u)
            solution_dict[u] = []
            V_partition = [0 if i in get_ideal(G, V, u) else 1 for i in range(N)]
            s, ideal_partition = partition_ideal(G, W, V_partition, u)
            precalc[u] = (ideal_size, ideal, s[0], ideal_partition); sizes[u] = ideal_size
            ideals[u] = ideal
            
        V, aux = visit(G, V, u)
        ans.append(u); tot += aux
        
    ans_ = list(itertools.chain.from_iterable(([j,i] if j != [] else [i] for i,j in solution_dict.items())))
    ans = [i[0] if type(i) == list else i for i in ans_]
    sizes['out'] = len(G.nodes()) - sum(V)
    return ans, solution_dict, sizes, ideals

## get_dp_table:

- Input: Grafo tipo árbol (G), lista de nodos visitados (V), Cantidad de muestras (k), nodo sobre cuyo ideal se trabajará (s), cantidad de valores max_size a agrupar (mult).
- Output: Diccionario de posibilidades válidas por nodo y diccionario de combinaciones en hijos que las generan

In [14]:
# Esta función usa programación dinámica para encontrar las muestras

def get_dp_table(G, V, k, s, mult):
    
    N = len(V)

    order = get_ideal(G, V, s)[::-1]  # Lista de nodos en ideal a trabajar desde más profundo a menos profundo
    size = len(order)
    
    # inicializamos tablas

    # Combinaciones de (número de muestras, nodos sin muestrear, ideal máximo, ideal mínimo) en el ideal de cada u
    possibilities = {u: [] for u in order}
    
    # Punteros óptimos para generar cada posibilidad en la tabla (posibilities) para cada u
    next_dicts = {u: {} for u in order}

    for u in order: # Para cada nodo de más a menos profundo

        new_possibilities = {}; new_next = {}  # Subtablas del nodo u (al final se pasan a tablas finales)
        predecessors = [v for v in G.predecessors(u) if not V[v]]  # nodos hijos en el árbol

        # calculamos cantidad de combinaciones en posibilidades de hijos
        # y generamos tabla (mul) para generar las combinaciones
        
        mul = [1 for v in predecessors]; combs = 1 if predecessors else 0
        for i in range(len(predecessors)):
            now = len(possibilities[predecessors[i]])
            for j in range(i):
                mul[j] *= now
            combs *= now
        
        for i in range(combs):  # por cada combinación de posibilidades

            indexes = [0 for v in predecessors]  # generamos la combinación
            for j in range(len(predecessors)):
                indexes[j] = (i // mul[j])
                i %= mul[j]
                
            # calculamos la posibilidad que genera esta combinación

            l = 0; size_left = 1; max_ideal = 0; min_ideal = N
            for j in range(len(predecessors)):
                pos = possibilities[predecessors[j]][indexes[j]]
                l += pos[0]
                size_left += pos[1]
                max_ideal = max(max_ideal, pos[2])
                min_ideal = min(min_ideal, pos[3])

            if l >= k:
                continue
                
            # Agregamos posibilidades

            if new_possibilities.get((l, max_ideal // mult)):
                prev = new_possibilities[(l, max_ideal // mult)]
                if size_left < prev[0] or (size_left == prev[0] and min_ideal > prev[2]):
                    new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
                    new_next[(l, max_ideal // mult)] = indexes
            else:
                new_possibilities[(l, max_ideal // mult)] = (size_left, max_ideal, min_ideal)
                new_next[(l, max_ideal // mult)] = indexes
                
            if l < k:
                if new_possibilities.get((l + 1, max(size_left, max_ideal) // mult)):
                    prev = new_possibilities[(l + 1, max(size_left, max_ideal) // mult)]
                    if 0 < prev[0] or (0 == prev[0] and min(size_left, min_ideal) > prev[2]):
                        new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
                        new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes
                else:
                    new_possibilities[(l + 1, max(size_left, max_ideal) // mult)] = (0, max(size_left, max_ideal), min(size_left, min_ideal))
                    new_next[(l + 1, max(size_left, max_ideal) // mult)] = indexes

        # Caso especial, si no hay hijos no hay combinaciones y agregamos posibilidades de muestrear o no
                    
        if combs == 0:
            new_possibilities[(0, 0)] = (1, 0, N)
            new_next[(0, 0)] = []
            new_possibilities[(1, 0)] = (0, 1, 1)
            new_next[(1, 0)] = []
        
        final_possibilities = set()
        for item in new_possibilities.items():
            o, b = item;
            a = (o[0], b[0], b[1], b[2])
            final_possibilities.add(a)
            next_dicts[u][a] = new_next[o]

        possibilities[u] = list(final_possibilities)

    return possibilities, next_dicts

## get_sample_from_table:

- Input: Diccionario de posibilidades por nodo (possibilities), diccionario de combinaciones que las generan (next_dicts), nodos visitados (V), número de muestras (k) y nodo raíz de la búsqueda actual (s).
- Output: Nodos donde muestrear y lista de tamaños de cada partición

In [17]:
def get_sample_from_table(G, possibilities, next_dicts, V, k, s):
    
    mv = 1e10; opt_pos = None
    for pos in possibilities[s]:
        if pos[0] == k and pos[1] == 0:
            now = pos[2]
            if now < mv:
                mv = now
                opt_pos = pos
                
    ans = []
    Q = deque([]); Q.append((s, opt_pos))
    while Q:
        u, pos = Q.popleft()

        if pos[1] == 0:
            ans.append(u)

        i = 0; indexes = next_dicts[u][pos]

        predecessors = [v for v in G.predecessors(u) if not V[v]]
        
        for v in predecessors:
            next_pos_v = possibilities[v][indexes[i]]
            Q.append((v, next_pos_v)); i += 1

    N = len(V); V_ = [0] * N; sizes = []
    for u in ans[::-1]:
        V_, x = visit(G, V_, u)
        sizes.append(x)
    
    return ans, sizes

In [ ]:
import random
from collections import deque, defaultdict

class DLNode:
    def __init__(self, v, nxt, prv):
        self.v = v
        self.next = nxt
        self.back = prv

class DList:
    
    MXNODES = 10000000
    nodes = [None for i in range(MXNODES)]
    cnt = 0
    
    def __init__(self, arr):
        
        self.first = None
        self.last  = None
        
        for v in arr:
            DList.nodes[DList.cnt] = DLNode(v, None, self.last)
            if self.last:
                DList.nodes[self.last].next = DList.cnt
            self.last = DList.cnt; DList.cnt += 1
            if not self.first:
                self.first = self.last
    
    def reset(self):
        DList.cnt = 0
        DList.nodes = [None for i in range(MXNODES)]
    
    def append_right(self, v):
        
        DList.nodes[DList.cnt] = DLNode(v, None, self.last)
        if self.first:
            DList.nodes[self.last].next = DList.cnt
        else:
            self.first = DList.cnt
        self.last = DList.cnt; DList.cnt += 1
        
    def append_left(self, v):
        
        DList.nodes[DList.cnt] = DLNode(v, self.first, None)
        if self.first:
            DList.nodes[self.first].back = DList.cnt
        else:
            self.last = DList.cnt
        self.first = DList.cnt; DList.cnt += 1
        
    def extend_right(self, first, last):
        
        DList.nodes[self.last].next = first
        DList.nodes[first].back = self.last
        self.last = last
            
    def extend_left(self, first, last):
        
        DList.nodes[self.first].back = last
        DList.nodes[last].next = self.first
        self.first = first
        
    def remove_right(self, pos):
        
        self.last = pos
        DList.nodes[self.last].next = None
            
    def remove_left(self, pos):
        
        self.first = pos
        DList.nodes[self.first].back = None
        
    def remove_pos(self, pos):
        
        if self.first == pos:
            self.first = DList.nodes[pos].next
        if self.last == pos:
            self.last = DList.nodes[pos].back
        l = DList.nodes[pos].back; r = DList.nodes[pos].next
        if l is not None:
            DList.nodes[l].next = r
        if r is not None:
            DList.nodes[r].back = l
            
    def pop_left(self):
        
        if self.first is not None:
            self.first = DList.nodes[self.first].next
        if self.first is not None:
            DList.nodes[self.first].back = None
    
    def pop_right(self):
        
        if self.last is not None:
            self.last = DList.nodes[self.last].back
        if self.last is not None:
            DList.nodes[self.last].next = None
        

In [ ]:

def get_optimal_function(T, N, debug=False):
    
    order = [u for u in nx.topological_sort(T)][::-1]
    S = [DList([]) for i in range(N)]
    P = [0 for i in range(N)]

    F = {e: None for e in T.edges()}

    for u in order:  ## Generamos la extensión en orden buttom-up

        if debug:
            print(f"Node {u}")

        children = [v for v in T.successors(u)]

        if debug:
            print(f"children: {children}")
        if len(children) == 0:
            continue

        if len(children) == 1:  ## En caso de un sólo hijo

            v = children[0]

            # Buscamos el menor valor libre
            i = S[v].first
            free = 1
            while i:
                if DList.nodes[i].v > free:
                    break
                free = DList.nodes[i].v + 1
                i = DList.nodes[i].next

            S[u].append_right(free)

            F[(u, v)] = free

            if i is not None:  # Sólo nos quedamos con los valores expuestos
                S[u].extend_right(i, S[v].last)
            continue

        for v in children:
            P[v] = S[v].first

        # Buscamos l2 iterando coordinadamente

        l2 = -1; active = set(children); last_erased = None
        while len(active) > 1:
            l2 += 1

            to_erase = []
            for v in active:
                while P[v] is not None and DList.nodes[P[v]].v <= l2:
                    P[v] = DList.nodes[P[v]].next
                if P[v] is None:
                    to_erase.append(v)

            for v in to_erase:
                last_erased = v
                active.remove(v)

        i1 = None  # hijo de secuencia más grande
        if len(active) == 0:
            i1 = last_erased
        else:
            i1 = next(iter(active))

        children = [i1] + [ch for ch in children if ch != i1]

        # Generamos las listas ordenadas L[1:l2 + 1]

        L = [DList(children.copy())] + [None for i in range(l2)]
        L_ = {v: [] for v in children}

        it = L[0].first
        while it is not None:
            v = DList.nodes[it].v
            L_[v].append(it)
            it = DList.nodes[it].next

        last = {v: 0 for v in children}
        M = [{v: 0 for v in children}] + [None for i in range(l2)]
        C = [len(children)] + [0 for i in range(l2)]

        for v in children:
            P[v] = S[v].first

        for i in range(1, l2 + 1):    
            to_erase = set(); has_i = set()
            it = L[i - 1].first
            while it is not None:
                v = DList.nodes[it].v
                while P[v] is not None and DList.nodes[P[v]].v <= i:
                    if DList.nodes[P[v]].v == i:
                        has_i.add(v)
                    P[v] = DList.nodes[P[v]].next
                if P[v] is None:
                    to_erase.add(v)
                it = DList.nodes[it].next

            L_p = []; L_m = []
            it = L[i - 1].first
            while it is not None:
                v = DList.nodes[it].v
                if v in has_i:
                    L_p.append(v)
                elif v not in to_erase:
                    L_m.append(v)
                it = DList.nodes[it].next

            C[i] = 0
            L[i] = DList(L_p + L_m)
            it = L[i].first
            while it is not None:
                v = DList.nodes[it].v
                L_[v].append(it)
                it = DList.nodes[it].next
                C[i] += 1

            M[i] = {v: 0 for v in children}
            it = L[i].first
            while it is not None:
                v = DList.nodes[it].v
                M[i][v] = last[v]
                if v in has_i:
                    last[v] = i
                it = DList.nodes[it].next

        for v in children:
            P[v] = S[v].first
            nxt = None if P[v] is None else DList.nodes[P[v]].next
            while (P[v] is not None) and (nxt is not None) and DList.nodes[nxt].v <= l2:
                P[v] = nxt
                nxt = None if P[v] is None else DList.nodes[P[v]].next

        if debug:
            print(f"l2: {l2}")

        U = deque([])

        G = {v: 0 for v in children}

        p_i1 = P[i1]
        if (p_i1 is not None) and DList.nodes[p_i1].v == l2:
            p_i1 = DList.nodes[p_i1].next

        curr = l2; lst = l2; last_i1 = l2; cnt_0 = len(children)
        active = set()
        for v in children:
            if P[v] is not None:
                active.add(v)

        while curr > 0 or cnt_0 > 0:

            cnt = 0
            for v in active:
                if DList.nodes[P[v]].v == curr:
                    cnt += 1

            if debug:
                print(f"curr: {curr}, cnt: {cnt}, cnt_0: {cnt_0}")

            if cnt == 0 and lst > curr:
                for x in range(max(curr, 1), lst):
                    U.append(x)
            lst = curr
            if cnt <= 1 and curr != 0:
                to_remove = []
                for v in active:
                    if DList.nodes[P[v]].v == curr:
                        P[v] = DList.nodes[P[v]].back
                        if P[v] is None:
                            to_remove.append(v)
                for v in to_remove:
                    active.remove(v)
                curr -= 1
                continue

            w = None
            if U:
                w = U.pop()
            else:
                while p_i1 is not None:
                    if last_i1 + 1 < DList.nodes[p_i1].v:
                        break
                    last_i1 = DList.nodes[p_i1].v
                    p_i1 = DList.nodes[p_i1].next
                w = last_i1 + 1
                last_i1 += 1

            best_j = None
            if w <= l2:      
                flag = False
                if L[w].first is not None:
                    m = M[w][DList.nodes[L[w].first].v]
                    if C[w] == C[m]:
                        best_j = DList.nodes[L[w].first].v
                    elif C[m + 1] == C[w]:
                        best_j = DList.nodes[L[m].first].v
                    else:
                        flag = True

                if L[w].first is None or flag == True:
                    for i in range(w):
                        if C[i] > C[w] and C[i + 1] == C[w]:
                            best_j = DList.nodes[L[i].first].v
            else:
                if S[i1].first is not None and DList.nodes[S[i1].first].v < w:
                    best_j = i1
                else:
                    flag = False
                    if L[l2].first is not None:
                        m = M[l2][DList.nodes[L[l2].first].v]
                        if C[l2] == C[m]:
                            best_j = DList.nodes[L[l2].first].v
                        elif C[m + 1] == C[l2]:
                            best_j = DList.nodes[L[m].first].v
                        else:
                            flag = True

                    if L[l2].first is None or flag == True:
                        for i in range(l2):
                            if C[i] > C[l2] and C[i + 1] == C[l2]:
                                best_j = DList.nodes[L[i].first].v

            if debug:
                print(f"best_j: {best_j}, w: {w}, active: {len(active)}")

            to_add = deque([])
            while P[best_j] is not None and DList.nodes[P[best_j]].v <= w:
                v_ = DList.nodes[P[best_j]].v
                if v_ > curr:
                    to_add.append(v_)
                P[best_j] = DList.nodes[P[best_j]].next
            while to_add:
                U.append(to_add.pop())

            mxx = None
            if P[best_j] is None:
                if best_j != i1:
                    for i in range(last[best_j] + 1):
                        C[i] -= 1
                else:
                    for i in range(l2 + 1):
                        C[i] -= 1
                S[best_j] = DList([])
                mxx = l2
            else:
                S[best_j].remove_left(P[best_j])
                mxx = min(l2, DList.nodes[P[best_j]].v - 1)

            ll = len(L_[best_j]); mxx = min(mxx, ll - 1)
            while mxx >= 0 and L_[best_j][mxx] != -1:
                L[mxx].remove_pos(L_[best_j][mxx])
                L_[best_j][mxx] = -1
                M[mxx][best_j] = -1
                mxx -= 1

            if G[best_j] == 0:
                cnt_0 -= 1
            G[best_j] = w
            if best_j in active:
                active.remove(best_j)

        for k, v in G.items():
            F[(u, k)] = v

        for v in children:
            S[v].append_left(G[v])

        active = set(children)
        while len(active) > 1:

            id_ = None; mn = 1e9
            for v in active:
                if DList.nodes[S[v].first].v < mn:
                    mn = DList.nodes[S[v].first].v
                    id_ = v

            S[u].append_right(mn)
            S[id_].pop_left()
            if S[id_].first is None:
                active.remove(id_)

        i1 = next(iter(active))
        S[u].extend_right(S[i1].first, S[i1].last)
        
    print("Optimal Steps:", max(F.values()))
        
    return F


In [ ]:

def get_optimal_K_function(T, K, N, debug=False):
    
    order = [u for u in nx.topological_sort(T)][::-1]
    S = [DList([]) for i in range(N)]
    P = [0 for i in range(N)]

    F = {e: None for e in T.edges()}

    for u in order:  ## Generamos la extensión en orden buttom-up

        if debug:
            print(f"Node {u}")

        children = [v for v in T.successors(u)]

        if debug:
            print(f"children: {children}")
        if len(children) == 0:
            continue

        if len(children) == 1:  ## En caso de un sólo hijo

            v = children[0]

            # Buscamos el menor valor libre
            i = S[v].first
            free = 1
            while i:
                if DList.nodes[i].v[0] > free:
                    break
                elif DList.nodes[i].v[1] < K:
                    break
                free = DList.nodes[i].v[0] + 1
                i = DList.nodes[i].next

            if i and DList.nodes[i].v[0] == free:
                DList.nodes[i].v = (DList.nodes[i].v[0], DList.nodes[i].v[1] + 1)
                S[u].append_right(DList.nodes[i].v)
                i = DList.nodes[i].next
            else:
                S[u].append_right((free, 1))

            F[(u, v)] = free
            F[(v, u)] = free

            if i is not None:  # Sólo nos quedamos con los valores expuestos
                S[u].extend_right(i, S[v].last)
            continue

        for v in children:
            P[v] = S[v].first

        # Buscamos l2 iterando coordinadamente

        l2 = -1; active = set(children); last_erased = None
        while len(active) > 1:
            l2 += 1

            to_erase = []
            for v in active:
                while P[v] is not None and DList.nodes[P[v]].v[0] <= l2:
                    P[v] = DList.nodes[P[v]].next
                if P[v] is None:
                    to_erase.append(v)

            for v in to_erase:
                last_erased = v
                active.remove(v)

        i1 = None  # hijo de secuencia más grande
        if len(active) == 0:
            i1 = last_erased
        else:
            i1 = next(iter(active))

        children = [i1] + [ch for ch in children if ch != i1]

        # Generamos las listas ordenadas L[1:l2 + 1]

        L = [DList(children.copy())] + [None for i in range(l2)]
        L_ = {v: [] for v in children}

        it = L[0].first
        while it is not None:
            v = DList.nodes[it].v
            L_[v].append(it)
            it = DList.nodes[it].next

        last = {v: 0 for v in children}
        M = [{v: 0 for v in children}] + [None for i in range(l2)]
        C = [len(children)] + [0 for i in range(l2)]

        for v in children:
            P[v] = S[v].first
            
        for i in range(1, l2 + 1):    
            to_erase = set(); has_i = set()
            it = L[i - 1].first
            while it is not None:
                v = DList.nodes[it].v
                while P[v] is not None and DList.nodes[P[v]].v[0] <= i:
                    if DList.nodes[P[v]].v[0] == i:
                        has_i.add(v)
                    P[v] = DList.nodes[P[v]].next
                if P[v] is None:
                    to_erase.add(v)
                it = DList.nodes[it].next

            L_p = []; L_m = []
            it = L[i - 1].first
            while it is not None:
                v = DList.nodes[it].v
                if v in has_i:
                    L_p.append(v)
                elif v not in to_erase:
                    L_m.append(v)
                it = DList.nodes[it].next

            C[i] = 0
            L[i] = DList(L_p + L_m)
            it = L[i].first
            while it is not None:
                v = DList.nodes[it].v
                L_[v].append(it)
                it = DList.nodes[it].next
                C[i] += 1

            M[i] = {v: 0 for v in children}
            it = L[i].first
            while it is not None:
                v = DList.nodes[it].v
                M[i][v] = last[v]
                if v in has_i:
                    last[v] = i
                it = DList.nodes[it].next

        for v in children:
            P[v] = S[v].first
            nxt = None if P[v] is None else DList.nodes[P[v]].next
            while (P[v] is not None) and (nxt is not None) and DList.nodes[nxt].v[0] <= l2:
                P[v] = nxt
                nxt = None if P[v] is None else DList.nodes[P[v]].next

        if debug:
            print(f"l2: {l2}")

        U = deque([])

        G = {v: 0 for v in children}

        p_i1 = P[i1]
        while (p_i1 is not None) and DList.nodes[p_i1].v[0] <= l2:
            p_i1 = DList.nodes[p_i1].next

        curr = l2; lst = l2; last_i1 = l2 + 1; last_i1_cnt = 0; cnt_0 = len(children)
        active = set()
        for v in children:
            if P[v] is not None:
                active.add(v)

        while curr > 0 or cnt_0 > 0:

            cnt = 0
            for v in active:
                if DList.nodes[P[v]].v[0] == curr:
                    cnt += DList.nodes[P[v]].v[1]

            if debug:
                print(f"curr: {curr}, cnt: {cnt}, cnt_0: {cnt_0}")

            if cnt < K and lst > curr:
                for x in range(max(curr, 1), lst):
                    if x == curr:
                        U.append((x, K - cnt))
                    else:
                        U.append((x, K))
            lst = curr
            if cnt <= K and curr != 0:
                to_remove = []
                for v in active:
                    if DList.nodes[P[v]].v[0] == curr:
                        P[v] = DList.nodes[P[v]].back
                        if P[v] is None:
                            to_remove.append(v)
                for v in to_remove:
                    active.remove(v)
                curr -= 1
                continue

            w = None
            
            if U:
                w, cc = U.pop()
                if cc > 1:
                    U.append((w, cc - 1))
            else:
                while p_i1 is not None:
                    if last_i1 == DList.nodes[p_i1].v[0] and DList.nodes[p_i1].v[1] < K:
                        w = last_i1
                        last_i1_cnt += 1
                        if last_i1_cnt + DList.nodes[p_i1].v[1] == K:
                            last_i1 += 1
                            last_i1_cnt = 0
                            p_i1 = DList.nodes[p_i1].next
                        break
                    elif last_i1 == DList.nodes[p_i1].v[0] and DList.nodes[p_i1].v[1] == K:
                        last_i1 += 1
                        last_i1_cnt = 0
                        p_i1 = DList.nodes[p_i1].next
                    elif last_i1 < DList.nodes[p_i1].v[0] and last_i1_cnt < K:
                        w = last_i1
                        last_i1_cnt += 1
                        break
                    elif last_i1 < DList.nodes[p_i1].v[0] and last_i1_cnt == K:
                        last_i1 += 1
                        last_i1_cnt = 0
                if w is None:
                    w = last_i1
                    last_i1_cnt += 1
                    if last_i1_cnt == K:
                        last_i1 += 1
                        last_i1_cnt = 0
            
            best_j = None
            if w <= l2:      
                flag = False
                if L[w].first is not None:
                    m = M[w][DList.nodes[L[w].first].v]
                    if C[w] == C[m]:
                        best_j = DList.nodes[L[w].first].v
                    elif C[m + 1] == C[w]:
                        best_j = DList.nodes[L[m].first].v
                    else:
                        flag = True

                if L[w].first is None or flag == True:
                    for i in range(w):
                        if C[i] > C[w] and C[i + 1] == C[w]:
                            best_j = DList.nodes[L[i].first].v
            else:
                if S[i1].first is not None and DList.nodes[S[i1].first].v[0] < w:
                    best_j = i1
                else:
                    flag = False
                    if L[l2].first is not None:
                        m = M[l2][DList.nodes[L[l2].first].v]
                        if C[l2] == C[m]:
                            best_j = DList.nodes[L[l2].first].v
                        elif C[m + 1] == C[l2]:
                            best_j = DList.nodes[L[m].first].v
                        else:
                            flag = True

                    if L[l2].first is None or flag == True:
                        for i in range(l2):
                            if C[i] > C[l2] and C[i + 1] == C[l2]:
                                best_j = DList.nodes[L[i].first].v

            if debug:
                print(f"best_j: {best_j}, w: {w}, active: {len(active)}")
            
            to_add = deque([])
            if P[best_j] is None:
                P[best_j] = S[best_j].first
            while P[best_j] is not None and DList.nodes[P[best_j]].v[0] < w:
                v_ = DList.nodes[P[best_j]].v
                if v_[0] > curr:
                    to_add.append(v_)
                P[best_j] = DList.nodes[P[best_j]].next
            while to_add:
                U.append(to_add.pop())

            mxx = None
            if P[best_j] is None:
                if best_j != i1:
                    for i in range(last[best_j] + 1):
                        C[i] -= 1
                else:
                    for i in range(l2 + 1):
                        C[i] -= 1
                S[best_j] = DList([])
                mxx = l2
            else:
                S[best_j].remove_left(P[best_j])
                mxx = min(l2, DList.nodes[P[best_j]].v[0] - 1)

            ll = len(L_[best_j]); mxx = min(mxx, ll - 1)
            while mxx >= 0 and L_[best_j][mxx] != -1:
                L[mxx].remove_pos(L_[best_j][mxx])
                L_[best_j][mxx] = -1
                M[mxx][best_j] = -1
                mxx -= 1

            if G[best_j] == 0:
                cnt_0 -= 1
            G[best_j] = w
            if best_j in active:
                active.remove(best_j)

        for k, v in G.items():
            F[(u, k)] = v
            F[(k, u)] = v

        for v in children:
            if S[v].first is not None and DList.nodes[S[v].first].v[0] == G[v]:
                DList.nodes[S[v].first].v = (DList.nodes[S[v].first].v[0], DList.nodes[S[v].first].v[1] + 1)
            else:
                S[v].append_left((G[v], 1))

        active = set(children)
        while len(active) > 1:

            id_ = set(); mn = 1e9; cntt = 0
            for v in active:
                if debug:
                    print(v, DList.nodes[S[v].first].v)
                if DList.nodes[S[v].first].v[0] < mn:
                    mn = DList.nodes[S[v].first].v[0]
                    cntt = DList.nodes[S[v].first].v[1]
                    id_ = set([v])
                elif DList.nodes[S[v].first].v[0] == mn:
                    cntt += DList.nodes[S[v].first].v[1]
                    id_.add(v)
                    
            if debug:
                print(mn, cntt)

            S[u].append_right((mn, cntt))
            if cntt > K:
                print("FAILED", K, cntt)
                return 0
            for v in id_:
                S[v].pop_left()
                if S[v].first is None:
                    active.remove(v)

        if active:
            i1 = next(iter(active))
            S[u].extend_right(S[i1].first, S[i1].last)
        
    print("Optimal Steps:", max(F.values()))
        
    return F


In [ ]:
def simulate(G, N, F, s_init, nodes):
    
    GR = G.reverse()
    
    E = G.edges()
    
    k_sum = 0
    T = []
    
    for r in nodes:
        CV = [0 for u in range(N)]

        CV[r] = 1
        Q = deque([]); Q.append(r)
        while Q:
            u = Q.popleft()
            for v in G.predecessors(u):
                if not CV[v]:
                    CV[v] = 1
                    Q.append(v)

        V = [0 for u in range(N)]

        s = s_init; ks = []
        for t in range(100):

            P = []

            mx = 0
            Q = deque([s]); visited = set([s])
            while Q:
                u = Q.popleft()
                for v in G.successors(u):
                    if v not in visited and not V[v]:
                        if F[(u, v)] == mx:
                            if (u, v) in E:
                                P.append(v)
                            else:
                                P.append(u)
                        if F[(u, v)] > mx:
                            mx = F[(u, v)]
                            if (u, v) in E:
                                P = [v]
                            else:
                                P = [u]
                        visited.add(v)
                        Q.append(v)

            ks.append(len(P)); k_sum += len(P)

            for u in P[::-1]:
                if CV[u]:
                    I = set(get_ideal(GR, V, u)[::-1])
                    V = [not (v in I) for v in range(N)]
                    s = u; break
                else:
                    V, _ = visit(GR, V, u)

            size = N - sum(V)

            if size == 1:
                T.append(t + 1)
                print(f"Result {r}: {T[-1]}    acc: {sum(T) / (len(T))}    ks: {ks}")
                break
            if t == 99:
                print(f"{r}!!!")

    print(sum(T) / len(T), max(T), k_sum)
    
    return T
